<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li></ul></div>

# Description

We want to: 
- build a good set of functions to compute the size control
- run a sensitivity analysis of the size control as a function of the mass scaling and the relative degradation rate

The parameters that we expect to have a significant impact are: 
- the type of transition
- the rate of transition `k`
- the degradation ratio `epsilon`

The way we do it is: 

In a script:
- Run simulations for a single cell (I don't think it's useful to use populations in this case)
- Save simulation results
- Compute slopes for the different metrics we plot under `plot_size_control`

Here
- Analyze the results

# Load

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import os

import pickle

In [ ]:
EXP_DIR = "/Users/lucasfuentes/RB_model/data/size_control"

df_exp = pd.read_csv(os.path.join(EXP_DIR, "results.csv"), index_col = 0)

plot_size_control(df_exp, "/Users/lucasfuentes/RB_model/figs")

def plot_size_control(df_exp, save_dir):
    slopes_to_plot = ["slopes_G1_growth", "slopes_G1_length", "slopes_G1_delta"]
    
    for transition in ["size", "RBc"]:
        
        df_exp_ = df_exp.loc[df_exp["transition"] == transition]
        uniq_k = df_exp["k_trans"].unique()
        
        fig, ax = plt.subplots(layout='constrained',figsize=(10, 4 * uniq_k.shape[0]))
        subfigs = fig.subfigures(uniq_k.shape[0], 1)
        
        for (i,k) in enumerate(uniq_k):
            
            ax = subfigs[i].subplots(1, len(slopes_to_plot))
            subfigs[i].suptitle(f"k={k}")
            
            # plot the slopes as a function of epsilon, for different values of delta
            
            df_ = df_exp_.loc[df_exp_["k_trans"] == k].copy()
            uniq_d = df_["delta"].unique()

            for d in uniq_d:
                df_tmp = df_.loc[df_["delta"] == d]

                for (j, s) in enumerate(slopes_to_plot):

                    ax[j].plot(df_tmp["epsilon"], df_tmp[s], "o--", label=f"delta={d}")

                    ax[j].set_ylabel(s)

                    ax[j].set_xlabel("epsilon")

            ax[2].legend()
            
#             subfigs[1].tight_layout()

            # plt.legend(loc="outside")      
#         fig.tight_layout()
        fig.suptitle(f"transition = {transition}")
        plt.savefig(os.path.join(save_dir, f"{transition}.pdf"), dpi=200)
        plt.close()
        
    return